# Converting Text to N dimension Vector By using Bag Of Words Approach


In [51]:
# Load text First
#import all neccesary lib
%matplotlib inline 


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer


#using the SQLite Table to read data
con = sqlite3.connect(r'E:\Applied AI\AppliedAICoding\Dataset\amazon-fine-food-reviews\database.sqlite')

#filtering only positive and negative reviews i.e.
#nottaking into consideration those reiviews with score=3
filtered_data =pd.read_sql_query("""
Select * from Reviews Where Score !=3""",con)

print(filtered_data.shape)


(525814, 10)


In [52]:
# Give review with Score>3 a positive rating , and reviews with a score<3 a negative rating.

def partition(x):
    if x<3:
        return 'negative'
    return 'positive'

# changing reviews with score less than 3 to be positive and vice - versa
'''
Actual code 
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
'''

filtered_data['Score'] = filtered_data['Score'].map(partition)




In [53]:
print(filtered_data.shape)# no rows and column
filtered_data.head()


(525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [54]:
display=pd.read_sql_query("""
Select * from Reviews where score !=3 and UserId="AR5J8UI46CURR" Order by productID""",con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [55]:
#Sorting data according to productId in ascending order

#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
sorted_data= filtered_data.sort_values('ProductId',axis=0,ascending=True, inplace=False, kind= 'quicksort',na_position='last')
sorted_data.shape

(525814, 10)

In [56]:
#Deduplication of entries
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop_duplicates.html
final = sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"} , keep='first', inplace=False)
final.shape

(364173, 10)

In [57]:
#checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

In [58]:
display = pd.read_sql_query("""
Select * from Reviews
where Score !=3 and id=44737 or id=64422 order by productID""",con)


#or
'''Select * from Reviews where HelpfulnessNumerator>HelpfulnessDenominator'''
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [59]:

final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

# Text Preprocessing: Stemming, stop-word removal and Lemmatization.

In [60]:
# removing Html Tags from TEXT
import re
# find sentences containing HTML tags
#http://www.pythonforbeginners.com/regex/regular-expressions-in-python
i=0;
for sent in final['Text'].values:
    if(len(re.findall('<.*?>',sent))):
        print(i)
        print(sent)
        break
    i+=1

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [66]:
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop= set(stopwords.words('english')) # set of english stop words
sno = nltk.stem.SnowballStemmer('english') # intialize the snowball stemmer


def cleanhtml(sentence):
    cleanr= re.compile('<.*?>')
    cleantext=re.sub(cleanr,' ',sentence)
    return cleantext

def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned

print(stop)
print('************************************')
print(sno.stem('tasty'))

{"hasn't", 'just', 'where', "it's", 'other', "aren't", 'been', 'was', 'now', 'you', 'after', 'very', 'himself', 'again', 'had', 'we', 'into', 'being', 'his', 'him', 'themselves', 'they', 'out', 'mightn', 'this', 'a', 'hers', "you're", 've', 'won', 'the', "doesn't", 'on', 'whom', 'both', 'some', 'against', 'which', 'up', 'while', 'until', 'own', 'how', 'be', 'ours', 'from', 'any', 'it', "she's", 'that', "isn't", 'above', 'is', 'them', 'by', 'or', 'its', 'our', 'same', 'all', "that'll", 'weren', 'in', 'there', 'more', 'here', 'wasn', 'over', 'at', 'shouldn', "mustn't", 'an', 'such', 'theirs', 'of', 'no', 'than', 'd', 'she', 'itself', 'your', 'but', 'through', 'nor', 'too', "should've", 'so', 'down', 'few', 'doesn', "haven't", 'herself', 'if', 'having', "couldn't", 'o', 'ain', 'hasn', 'only', 'about', 'below', 's', 'm', 'and', 'have', 'off', 'before', 'mustn', 'because', "wouldn't", 'should', 'he', 'will', 'aren', 'those', 'her', "you'll", 'for', 'needn', "needn't", 'am', "you've", 'shan'

In [68]:
#Code for implementing step-by-step the checks mentioned in the pre-processing phase
# this code takes a while to run as 300k senetences
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) #remove HTML tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.lower() not in stop):
                s=(sno.stem(cleaned_words.lower())).encode('utf8')
                filtered_sentence.append(s)
                if(final['Score'].values)[i]=='positive':
                    all_positive_words.append(s) #list of all words used to describe positive reviews
                if(final['Score'].values)[i]=='negative':
                    all_negative_words.append(s) #List of all words used to describe negative reviews
                else:
                    continue
            else:
                continue
    #print(filtered_sentence)
    
    
    str1=b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    final_string.append(str1)
    i+=1
     
print(i)

KeyboardInterrupt: 

In [ ]:
final['CleanedText']=final_string # adding  a column of CleanedText which displays the data after pre-processing of the review 

In [ ]:
final.head(3) # below  the processed review can be seen in the CleanedText Column 


In [ ]:
# store final table into sqlite table for future
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html
conn = sqlite3.connect('final.sqlite')
c= conn.cursor()
conn.text_factory = str
final.to_sql('Reviews',conn,flavor=None, schema=None, if_exists='replace', index=True,index_label=None,chunksize=None,dtype=None)

# Bag of Words (BoW)

In [69]:
con = sqlite3.connect(r'E:\Applied AI\AppliedAICoding\Dataset\amazon-fine-food-reviews\chkpoint\final.sqlite')

#filtering only positive and negative reviews i.e.
#nottaking into consideration those reiviews with score=3
final =pd.read_sql_query("""
Select * from Reviews""",con)

In [70]:
final.head(3)

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh loud recit c...
1,138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read sendak book watch realli rosi movi...
2,138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'fun way children learn month year learn poem...


In [72]:
#BOW BAG Of Words
count_vect = CountVectorizer() #in scikit-learn
final_counts = count_vect.fit_transform(final['Text'].values)
                                        
type(final_counts)

scipy.sparse.csr.csr_matrix

In [73]:
final_counts.get_shape()

(364171, 115281)